In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pandas as pd
import os
from torch.utils.data import Dataset
from PIL import Image


In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset

class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))
        ])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.samples = []
        for cls_name in self.classes:
            class_folder = os.path.join(root_dir, cls_name)
            for fname in os.listdir(class_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_folder, fname)
                    label = self.class_to_idx[cls_name]
                    self.samples.append((img_path, label))
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((480, 480)),
    transforms.ToTensor()
])

dataset = BrainTumorDataset(
    root_dir="/kaggle/input/brain-tumor-mri-scans",
    transform=transform
)
dataloader = DataLoader(
    dataset, 
    batch_size=32, 
    shuffle=True
)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class LiteInceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LiteInceptionBlock, self).__init__()
        branch4 = out_channels // 4
        remain = out_channels - branch4
        branch1 = branch2 = branch3 = remain // 3
        self.branch1 = nn.Sequential(nn.Conv2d(in_channels, branch1, kernel_size=1, bias=False), nn.BatchNorm2d(branch1), nn.ReLU(inplace=True))
        self.branch2 = nn.Sequential(nn.Conv2d(in_channels, branch2, kernel_size=1, bias=False), nn.BatchNorm2d(branch2), nn.ReLU(inplace=True), DepthwiseSeparableConv(branch2, branch2, kernel_size=3, padding=1))
        self.branch3 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=1, padding=1), nn.Conv2d(in_channels, branch3, kernel_size=1, bias=False), nn.BatchNorm2d(branch3), nn.ReLU(inplace=True))
        self.branch4 = nn.Sequential(nn.Conv2d(in_channels, branch4, kernel_size=1, bias=False), nn.BatchNorm2d(branch4), nn.ReLU(inplace=True), DepthwiseSeparableConv(branch4, branch4, kernel_size=3, padding=1), DepthwiseSeparableConv(branch4, branch4, kernel_size=3, padding=1))
        self.out_bn = nn.BatchNorm2d(out_channels)
        self.out_relu = nn.ReLU(inplace=True)
    def forward(self, x):
        y1 = self.branch1(x)
        y2 = self.branch2(x)
        y3 = self.branch3(x)
        y4 = self.branch4(x)
        out = torch.cat([y1, y2, y3, y4], dim=1)
        out = self.out_bn(out)
        out = self.out_relu(out)
        return out

class LiteAuxHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(LiteAuxHead, self).__init__()
        self.conv1 = DepthwiseSeparableConv(in_channels, 32, kernel_size=3, padding=1)
        self.conv2 = DepthwiseSeparableConv(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, num_classes, kernel_size=1)
        self.dropout = nn.Dropout2d(0.1)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.AdaptiveAvgPool2d((1,1))
    def forward(self, x):
        x = self.conv1(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.dropout(x)
        x = self.conv3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return x

class LiteCustomInceptionNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(LiteCustomInceptionNet, self).__init__()
        self.backbone = nn.Sequential(nn.Conv2d(in_channels, 16, kernel_size=3, stride=2, padding=1, bias=False), nn.BatchNorm2d(16), nn.ReLU(inplace=True), DepthwiseSeparableConv(16, 32, kernel_size=3, stride=2, padding=1))
        self.incep1 = LiteInceptionBlock(32, 64)
        self.incep2 = LiteInceptionBlock(64, 128)
        self.incep3 = LiteInceptionBlock(128, 192)
        self.aux1 = LiteAuxHead(64, num_classes)
        self.aux2 = LiteAuxHead(128, num_classes)
        self.aux3 = LiteAuxHead(192, num_classes)
    def forward(self, x):
        x = self.backbone(x)
        x = self.incep1(x)
        out1 = self.aux1(x)
        x = self.incep2(x)
        out2 = self.aux2(x)
        x = self.incep3(x)
        out3 = self.aux3(x)
        return out1, out2, out3

In [4]:
from tqdm import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

feature_size = (120, 120)
num_classes = 4
model = LiteCustomInceptionNet(in_channels=3, num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for images, masks in progress_bar:
        images = images.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        out1, out2, out3 = model(images)
        loss1 = criterion(out1, masks)
        loss2 = criterion(out2, masks)
        loss3 = criterion(out3, masks)
        loss = 0.15 * loss1 + 0.35 * loss2 + loss3
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
    scheduler.step(avg_loss)
    

Epoch 1/20, Loss: 1.4086


Epoch 2/20, Loss: 0.9426


Epoch 3/20, Loss: 0.7940


Epoch 4/20, Loss: 0.6985


Epoch 5/20, Loss: 0.6480


...

Epoch 16/20, Loss: 0.3738


Epoch 17/20, Loss: 0.3523


Epoch 18/20, Loss: 0.3511


Epoch 19/20, Loss: 0.3224


Epoch 20/20, Loss: 0.3101


In [5]:
torch.save(model.state_dict(), "model.pth")

In [15]:
import torch
from torchvision import transforms
from PIL import Image

model = LiteCustomInceptionNet(in_channels=3, num_classes=4)
model.load_state_dict(torch.load("model.pth"))
model.eval()
model.to(device)
class_names = ["glioma", "healthy", "meningioma", "pituitary"]
inference_transform = transforms.Compose([
    transforms.Resize((480,480)),
    transforms.ToTensor()
])

def load_image_for_inference(img_path):
    image = Image.open(img_path).convert("RGB")
    image = inference_transform(image)
    return image.unsqueeze(0)

img_path = "/kaggle/input/brain-tumor-mri-scans/pituitary/0023.jpg"
input_tensor = load_image_for_inference(img_path).to(device)

with torch.no_grad():
    out1, out2, out3 = model(input_tensor)
    output = out3
    predicted_index = torch.argmax(output, dim=1).item()
    predicted_label = class_names[predicted_index]
    print("Predicted label:", predicted_label)

Predicted label: pituitary


<ipython-input-15-cb04b7995664>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))
